In [19]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict
import shutil
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()

True

# Dataset Description

In [15]:
import pandas as pd
from datasets import load_from_disk
from IPython.display import display, Markdown

# Define the file paths
output_dataset_dir = 'tax_law_brazil_cosit'
readme_path = f'{output_dataset_dir}/README.md'

# Load the dataset from disk
dataset_dict = load_from_disk(output_dataset_dir)

# Read the description and features from README.md
with open(readme_path, 'r', encoding='utf-8') as file:
    readme_content = file.read()

# Split the README content into description and features
description, features_section = readme_content.split("## Features", 1)

# Prepare the markdown content
markdown_content = []

markdown_content.append("# Dataset Description")
markdown_content.append(description.strip())

markdown_content.append("\n## Dataset Features")
markdown_content.append(features_section.strip())

# Print the splits
markdown_content.append("\n## Dataset Splits")
markdown_content.append(f"```python\n{dataset_dict}\n```")

# Print one row from the tax_law split
markdown_content.append("\n## Sample row from 'tax_law' split")
markdown_content.append(f"```python\n{dataset_dict['tax_law'][0]}\n```")

# Print one document from the corpus split
markdown_content.append("\n## Sample document from 'corpus' split")
markdown_content.append(f"#### {dataset_dict['corpus']['document_id'][0]}:\n{dataset_dict['corpus']['content'][0][:500]}...")

# Combine the markdown content into a single string
markdown_output = "\n".join(markdown_content)

# Display the markdown content
display(Markdown(markdown_output))


# Dataset Description
# Tax Law Brazil Cosit Dataset


Our dataset consists of a series of tax law questions related to legal entities. The questions were selected from a collection that is annually updated by the General Coordination of Taxation (Cosit). 
https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/declaracoes-e-demonstrativos/ecf/perguntas-e-respostas-pj-2023.pdf 
Accessed: 11/11/2023

## Dataset Features
- **question**: The tax-related question.
- **answer**: The official answer to the tax-related question.
- **reference**: The specific normative reference related to the question.
- **gold_passage**: The gold passage or specific reference text related to the question.
- **corpus_documents**: Content of the related documents in the corpus.

## Dataset Splits
```python
DatasetDict({
    tax_law: Dataset({
        features: ['question', 'answer', 'reference', 'gold_passage'],
        num_rows: 101
    })
    corpus: Dataset({
        features: ['document_id', 'content'],
        num_rows: 30
    })
})
```

## Sample row from 'tax_law' split
```python
{'question': 'Que pessoas jurídicas estão desobrigadas de apresentar a ECF?', 'answer': 'Estão desobrigadas de apresentar a ECF: I - as pessoas jurídicas optantes pelo Regime Especial Unificado de Arrecadação de Tributos e Contribuições devidos pelas Microempresas e Empresas de Pequeno Porte, instituído pela Lei Complementar nº 123, de 14 de dezembro de 2006 (Simples Nacional), por estarem obrigadas à apresentação de Declaração Anual do Simples Nacional - DASN; II - os órgãos públicos, as autarquias e as fundações públicas; e III - as pessoas jurídicas inativas, assim consideradas aquelas que não tenham efetuado qualquer atividade operacional, não operacional, patrimonial ou financeira, inclusive aplicação no mercado financeiro ou de capitais, durante todo o ano-calendário, as quais deverão cumprir as obrigações acessórias previstas na legislação específica.', 'reference': 'IN RFB nº 2.004, de 2021, art. 1º, § 1º;', 'gold_passage': 'Art. 1º A Escrituração Contábil Fiscal (ECF) será apresentada, a partir do ano-calendário de 2014, por todas as pessoas jurídicas, inclusive as equiparadas, de forma centralizada pela matriz, de acordo com as regras estabelecidas nesta Instrução Normativa.\r\n§ 1º A obrigatoriedade a que se refere o caput não se aplica:\r\nI - às pessoas jurídicas optantes pelo Regime Especial Unificado de Arrecadação de Tributos e Contribuições devidos pelas Microempresas e Empresas de Pequeno Porte (Simples Nacional), de que trata a Lei Complementar nº 123, de 14 de dezembro de 2006;\r\nII - aos órgãos públicos, às autarquias e às fundações públicas; e\r\nIII - às pessoas jurídicas inativas, assim consideradas aquelas que não tenham efetuado qualquer atividade operacional, não operacional, patrimonial ou financeira, inclusive aplicação no mercado financeiro ou de capitais, durante todo o ano-calendário, as quais devem cumprir as obrigações acessórias previstas na legislação específica.\r\n§ 2º Para as pessoas jurídicas que apuram o Imposto sobre a Renda das Pessoas Jurídicas (IRPJ) pela sistemática do lucro real, a ECF é o Livro de Apuração do Lucro Real a que se refere o inciso I do caput do art. 8º do Decreto-Lei nº 1.598, de 26 de dezembro de 1977.\r\n§ 3º No caso de pessoas jurídicas sócias ostensivas de Sociedades em Conta de Participação (SCP), a ECF deverá ser transmitida separadamente, para cada SCP, além da transmissão da ECF da sócia ostensiva.'}
```

## Sample document from 'corpus' split
#### Decreto-Lei nº 1.598, de 1977.txt:
DECRETO-LEI Nº 1.598, DE 26 DE DEZEMBRO DE 1977.

Vigência	
Altera a legislação do imposto sobre a renda.

        O PRESIDENTE DA REPÚBLICA , no uso das atribuições que lhe confere o artigo 55, item II, da Constituição, e tendo em vista a necessidade de adaptar a legislação do imposto sobre a renda às inovações da lei de sociedades por ações (Lei nº 6.404, de 15 de dezembro de 1976),

        DECRETA:

        Art 1º - O imposto sobre o lucro das pessoas jurídicas domiciliadas no País, inclusiv...

# Prompt Question Answer

In [20]:
retriever_template = """Use the following pieces of legal information from laws to answer the user's question.
If the answer is not clear in context, try to figure out by interpreting the information.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}

Question: {question}

Do not quote the "contextual information" provided in the answer, do not say "according to the information" or anything like that, use the information only to answer the question.
Only return the helpful answer below and nothing else.
REMEMBER: answer the question in portuguese.
Helpful answer:"""

prompt = ChatPromptTemplate.from_template(retriever_template)

# Models

In [21]:
models = ['mistralai/Mixtral-8x7B-Instruct-v0.1',
               'mistralai/Mistral-7B-Instruct-v0.2',
               'zero-one-ai/Yi-34B-Chat',
               'garage-bAInd/Platypus2-70B-instruct',
               'google/gemma-7b-it',
               'lmsys/vicuna-13b-v1.5',
               'lmsys/vicuna-7b-v1.5',
               'meta-llama/Llama-2-70b-chat-hf',
               'meta-llama/Llama-2-13b-chat-hf',
               'meta-llama/Llama-2-7b-chat-hf',
               'openchat/openchat-3.5-1210',
               'WizardLM/WizardLM-13B-V1.2',
               'Qwen/Qwen1.5-14B-Chat',
               'Qwen/Qwen1.5-72B-Chat',
               'upstage/SOLAR-10.7B-Instruct-v1.0',
               'meta-llama/Llama-3-70b-chat-hf',
               'mistralai/Mistral-7B-Instruct-v0.3',
               'mistralai/Mixtral-8x22B-Instruct-v0.1',
               'meta-llama/Llama-3-8b-chat-hf',
               'Qwen/Qwen2-72B-Instruct',
               'Qwen/Qwen1.5-110B-Chat',
               'teknium/OpenHermes-2p5-Mistral-7B',
               'openia/gpt-3.5-turbo'
               ]

# Answer Generation

In [26]:
import os

for model in models:
    if model == 'openai/gpt-3.5-turbo':
        llm = ChatOpenAI(
            temperature=0.1,
            model='gpt-3.5-turbo',
        )
    else:
        llm = ChatOpenAI(
            base_url="https://api.together.xyz/v1",
            temperature=0.1,
            api_key=os.getenv('TOGETHER_API_KEY'),
            model=model,
        )
    
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    data = []

    for row in tqdm(dataset_dict['tax_law'], desc=f"Processing with {model.split('/')[1]}"):
        question = row['question']
        gold_passage = row['gold_passage']
        answer = row['answer']
        model_answer = chain.invoke({'context': gold_passage, 'question': question})
        data.append([question, answer, model_answer])

    df_m = pd.DataFrame(columns=['Q', 'A', 'A_model'], data=data)
    df_m.to_csv(f'result_question_answer_generation/{model.split('/')[1]}_QAG.csv', index=False)

print('Finish')


Processing with gpt-3.5-turbo: 100%|██████████| 101/101 [00:00<00:00, 46470.46it/s]

Finish
